In [4]:
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import os
from tqdm import tqdm

In [5]:
tree = ET.parse('Задание2/masks.xml')
root = tree.getroot()

result_masks_directory = 'result/masks/'
result_photos_directory = 'result/images/'

In [6]:
image_list = []
for elem in root.findall('image'):
    image_list.append(elem)

for i in tqdm(range(len(image_list))):
    picture = f'Задание2/images/{image_list[i].get("name").split("/")[-1]}'

    with Image.open(picture) as mask_img:
        mask_img.load()
        draw = ImageDraw.Draw(mask_img)
        mask_img.convert('RGBA')
        mask_img.putalpha(0)
        for item in range(len(image_list[i])):
            subelem = image_list[i][item]
            if subelem.get('label') != 'Ignore':
                points = subelem.get('points').split(';')
                points = [(float(x), float(y)) for x, y in (point.split(',') for point in points)]
                draw.polygon(points, fill='magenta')
            else:
                points = subelem.get('points').split(';')
                points = [(float(x), float(y)) for x, y in (point.split(',') for point in points)]
                draw.polygon(points, fill='black')
            if not os.path.exists(result_masks_directory):
                os.makedirs(result_masks_directory)
            mask_img.save(f'{result_masks_directory}mask_{image_list[i].get("name").split("/")[-1]}.png')

    with Image.open(picture) as photo:
        photo.load()
        mask_img = mask_img.convert('RGBA')
        black_pixels = mask_img.load()
        width, height = mask_img.size
        for y in range(height):
            for x in range(width):
                if black_pixels[x, y] == (0, 0, 0, 255):
                    black_pixels[x, y] = (0, 0, 0, 0)
        photo.paste(mask_img, (0, 0), mask_img)
        if not os.path.exists(result_photos_directory):
            os.makedirs(result_photos_directory)
        photo.save(f'{result_photos_directory}photo_{image_list[i].get("name").split("/")[-1]}.png')

100%|██████████| 8/8 [00:21<00:00,  2.72s/it]
